### Step 1. 필요한 라이브러리 임포트

In [1]:
import pandas as pd
import re

### Step 2. [데이터](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv) 확인

- order_id 주문번호
- quantity 주문 개수
- item_name 주문한 상품명
- choice_description 선택사항
- item_price 가격

### Step 3. 데이터 불러오기


**read_csv를 통해 데이터를 올바른 형태로 불러온 뒤 chipo에 저장해주세요**  
data: chipotle.tsv

In [2]:
chipo = pd.read_csv('chipotle.tsv', sep = '\t')
chipo.info()
chipo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


### Step 4. 누락값 확인하기 
chipo에서 누락값이 존재하는지 확인해주세요

In [3]:
chipo.isnull().sum()

order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64

### Step 5. item_price에서 $를 없애고 float 형식으로 새롭게 저장해주세요.

In [4]:
item_strip = chipo['item_price'].apply(lambda x: x.strip('$'))
chipo['item_price'] = item_strip.apply(lambda x: float(x))
chipo.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98


### Step 6. 아이템별 주문 수를 출력해주세요

In [5]:
chipo['item_name'].value_counts()

Chicken Bowl                             726
Chicken Burrito                          553
Chips and Guacamole                      479
Steak Burrito                            368
Canned Soft Drink                        301
Steak Bowl                               211
Chips                                    211
Bottled Water                            162
Chicken Soft Tacos                       115
Chips and Fresh Tomato Salsa             110
Chicken Salad Bowl                       110
Canned Soda                              104
Side of Chips                            101
Veggie Burrito                            95
Barbacoa Burrito                          91
Veggie Bowl                               85
Carnitas Bowl                             68
Barbacoa Bowl                             66
Carnitas Burrito                          59
Steak Soft Tacos                          55
6 Pack Soft Drink                         54
Chips and Tomatillo Red Chili Salsa       48
Chicken Cr

### Step 7. 아이템별 주문 총량을 출력해주세요.

In [6]:
chipo.groupby('item_name')['quantity'].sum()

item_name
6 Pack Soft Drink                         55
Barbacoa Bowl                             66
Barbacoa Burrito                          91
Barbacoa Crispy Tacos                     12
Barbacoa Salad Bowl                       10
Barbacoa Soft Tacos                       25
Bottled Water                            211
Bowl                                       4
Burrito                                    6
Canned Soda                              126
Canned Soft Drink                        351
Carnitas Bowl                             71
Carnitas Burrito                          60
Carnitas Crispy Tacos                      8
Carnitas Salad                             1
Carnitas Salad Bowl                        6
Carnitas Soft Tacos                       40
Chicken Bowl                             761
Chicken Burrito                          591
Chicken Crispy Tacos                      50
Chicken Salad                              9
Chicken Salad Bowl                       123


### Step 8. 각 아이템의 가격을 구해서 unit_price 컬럼에 추가해주세요.
주문당 quantity가 1일때의 가격 (item_price / quantity)

In [7]:
chipo['unit_price'] = chipo['item_price'] / chipo['quantity']
chipo.head()

,order_id,quantity,item_name,choice_description,item_price,unit_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,2.39
1,1,1,Izze,[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49


### Step 9. 아이템별 개당 가격의 평균을 구하고 가장 비싼순으로 5개를 출력해주세요

In [8]:
chipo.groupby('item_name')['unit_price'].mean().sort_values(ascending = False)[0:5]

item_name
Carnitas Salad Bowl    11.056667
Steak Salad Bowl       11.027931
Barbacoa Salad Bowl    10.640000
Carnitas Bowl          10.315588
Steak Bowl             10.232559
Name: unit_price, dtype: float64

### Step 10. 한 주문에 20달러 이상 지불한 손님을 출력해주세요.

In [9]:
tt_purc = chipo.groupby('order_id')['item_price'].sum() >= 20
result = pd.merge(chipo, tt_purc, how='left', on='order_id')
final = result[result['item_price_y'] == True]
final

,order_id,quantity,item_name,choice_description,item_price_x,unit_price,item_price_y
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75,11.75,True
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25,9.25,True
35,18,1,Chicken Soft Tacos,"[Roasted Chili Corn Salsa, Rice]",8.75,8.75,True
36,18,1,Chicken Soft Tacos,"[Roasted Chili Corn Salsa, [Cheese, Lettuce]]",8.75,8.75,True
37,18,1,Chips and Guacamole,NaN,4.45,4.45,True
...,...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75,11.75,True
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75,11.75,True
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25,11.25,True
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75,8.75,True


### Step 11. 위 10번에서 주문량이 2개인 아이템중 가장 많이 팔린 아이템은 무엇인가요?
Chicken Bowl, Chicken Salad Bowl

In [16]:
final[final['quantity'] == 2]['item_name'].value_counts().head(1)

Chips and Guacamole    20
Name: item_name, dtype: int64

### Step 12. Veggie Salad Bowl은 몇 건 주문되었나요?

In [11]:
chipo[chipo['item_name'] == 'Veggie Salad Bowl'].shape[0]

18

### Step 13. 아이템명에 Chicken이 들어간 음식은 몇 건 주문되었나요?
**Hint: 정규표현식 사용**   
1560

In [12]:
chipo[chipo['item_name'].str.contains('Chicken')].shape[0]

1560

In [24]:
chipo['item_name'].apply(lambda x: re.search('Chicken', x)).count()

1560